In [22]:
# coding: utf-8
 
import urllib
from pytube import YouTube
from pprint import pprint
from lxml import etree
import sys, getopt
 
 
def getHtml(url):
    user_agent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.13 (KHTML, like Gecko) Chrome/24.0.1284.0 Safari/537.13'
    headers = {'User-Agent': user_agent}
    request = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(request)
    html = response.read()
    return html
 
 
def getUrl(html):
    global savepath
    global maxNumber
    global timeThreshold
    global cur_count
    global videoLists
    tree = etree.HTML(html)
    urllist = tree.xpath(u'//div[@class="thumb-wrapper"]/a/@href')
    # print urllist
    urllist_time = tree.xpath(u'//div[@class="thumb-wrapper"]/a/span/span/text()')
 
    baseurl = r'https://www.youtube.com'
    for (item_name, item_length) in zip(urllist, urllist_time):
        # print item_name
        # print item_length
        try:
            yt = YouTube(baseurl + item_name)
        except:
            print ("Some thing wrong about the authority")
 
        print("video name:" + yt.filename)
        print("video time:" + item_length)
        if yt.filename in videoLists:  # 文件已经存在
            print ("This video has been downloaded!")
        else:
            if checktime(item_length):
                video = yt.filter('mp4')[-1]
                print("Now is loading %s------------>" % yt.filename)
                #video.download(savepath)
                print("ttt----------"+video.url)
                print("--------------->%sVideo is loaded!" % yt.filename)
                #cur_count += 1
                #videoLists.append(yt.filename)
                #if cur_count >= maxNumber:  # 达到要求
                #    print('There are %d videos downloaded!This task is completed!' % maxNumber)
                    # TODO: if necessary, the videoLists can be logged
                #    sys.exit()
            else:
                print ('This video is too long and it will not be downloaded, just be ignored!')
    if urllist:
        getUrl(baseurl + urllist[0])  # 下一个页面
 
 
def checktime(timelength):
    global timeThreshold
    strs = timelength.split(':')
    time = int(strs[0]) * 60 + int(strs[1])
    if time < timeThreshold:
        return True
    else:
        return False
 
 
def usage():
    print ('''
    usage: python dl_youtube [option] [arg]
    options and args:
    -s      : download path
    -t      : time threshold of the video to be loaded, in seconds
    -u      : start url which to be crawled, it can be set more than one time
    -n      : when downloading is stop, i.e. how many videos will be downloaded, default is 10000.
    -h      : print this help message
    ''')
 
#python dl_youtube.py -n 10 -s D://MyDownloads -t 600 -u https://www.youtube.com/watch?v=TThzH_sJo6o
if __name__ == "__main__":
    start_urls = [
                    'https://www.youtube.com/watch?v=ykmvaJ2Cf3w'
                    #'https://www.youtube.com/watch?v=TThzH_sJo6o'
        #"https: // www.youtube.com / watch?v = TThzH_sJo6o"
                  ]
    videoLists = []  # 保存文件名，防止重复下载
    # 初始值
    savepath = r"/home/mycode/tmp"
    maxNumber = 10000
    timeThreshold =600
    cur_count = 0
 
#
#    opts, args = getopt.getopt(sys.argv[1:], 'hs:t:n:u:')
#    for op, value in opts:
#        if op == "-s":  # 下载路径，如默认 D://MyDownloads
#            savepath = value
#        elif op == '-t':  # 时常限制，默认240s
#            timeThreshold = int(value)
#        elif op == "-h":  # help
#           usage()
#            sys.exit()
#        elif op == '-n':
#            maxNumber = int(value)
#        elif op == '-u':  # 初始的搜索链接
#            start_urls.append(value)
#
    for item in start_urls:
       html = getHtml(item)
       getUrl(html)



URLError: <urlopen error [Errno 101] Network is unreachable>

In [1]:
# -*-coding:utf-8-*-

from bs4 import BeautifulSoup
import lxml
import queue as Queue
import requests
import re,os,sys,random
import threading
import logging
import json,hashlib,urllib
from requests.exceptions import ConnectTimeout,ConnectionError,ReadTimeout,SSLError,MissingSchema,ChunkedEncodingError
import random
 
# reload(sys)
# sys.setdefaultencoding('gbk')
 
# 日志模块
logger = logging.getLogger("AppName")
formatter = logging.Formatter('%(asctime)s %(levelname)-5s: %(message)s')
console_handler = logging.StreamHandler(sys.stdout)
console_handler.formatter = formatter
logger.addHandler(console_handler)
logger.setLevel(logging.INFO)
 
q = Queue.Queue()   # url队列
page_q = Queue.Queue()  # 页面

proxies = {
    'https': 'https://127.0.0.1:1080',
    'http': 'http://127.0.0.1:1080'
}

def downlaod(q,x,path):
    urlhash = "https://weibomiaopai.com/"
    try:
        html = requests.get(urlhash,proxies=proxies).text
    except SSLError:
        logger.info(u"网络不稳定 正在重试")
        html = requests.get(urlhash,proxies=proxies).text
    reg = re.compile(r'var hash="(.*?)"', re.S)
    result = reg.findall(html)
    hash_v = result[0]
    while True:
        data = q.get()
        url, name = data[0], data[1].strip().replace("|", "")
        file = os.path.join(path, '%s' + ".mp4") % name
        api = "https://steakovercooked.com/api/video/?cached&hash=" + hash_v + "&video=" + url
        api2 = "https://helloacm.com/api/video/?cached&hash=" + hash_v + "&video=" + url
        try:
            res = requests.get(api,proxies=proxies)
            result = json.loads(res.text)
        except (ValueError,SSLError):
            try:
                res = requests.get(api2,proxies=proxies)
                result = json.loads(res.text)
            except (ValueError,SSLError):
                q.task_done()
                return False
        vurl = result['url']
        logger.info(u"正在下载：%s" %name)
        try:
            r = requests.get(vurl,proxies=proxies)
        except SSLError:
            r = requests.get(vurl,proxies=proxies)
        except MissingSchema:
            q.task_done()
            continue
        try:
            with open(file,'wb') as f:
                f.write(r.content)
        except IOError:
            name = u'好开心么么哒 %s' % random.randint(1,9999)
            file = os.path.join(path, '%s' + ".mp4") % name
            with open(file,'wb') as f:
                f.write(r.content)
        logger.info(u"下载完成：%s" %name)
        q.task_done()
 
def get_page(keyword,page_q):
    while True:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0'}
        page = page_q.get()
        url = "https://www.youtube.com/results?sp=EgIIAg%253D%253D&search_query=" + keyword + "&page=" + str(page)
        try:
            html = requests.get(url, headers=headers, proxies=proxies).text
        except (ConnectTimeout,ConnectionError):
            print (u"不能访问youtube 检查是否已FQ")
            os._exit(0)
        reg = re.compile(r'"url":"/watch\?v=(.*?)","webPageType"', re.S)
        result = reg.findall(html)
        logger.info(u"第 %s 页" % page)
        for x in result:
            vurl = "https://www.youtube.com/watch?v=" + x
            try:
                res = requests.get(vurl,proxies=proxies).text
            except (ConnectionError,ChunkedEncodingError):
                logger.info(u"网络不稳定 正在重试")
                try:
                    res = requests.get(vurl,proxies=proxies).text
                except SSLError:
                    continue
            reg2 = re.compile(r"<title>(.*?)YouTube",re.S)
            name = reg2.findall(res)[0].replace("-","")
            if u'\u4e00' <= keyword <= u'\u9fff':
                q.put([vurl, name])
            else:
                # 调用金山词霸
                logger.info(u"正在翻译")
                url_js = "http://www.iciba.com/" + name
                html2 = requests.get(url_js,proxies=proxies).text
                soup = BeautifulSoup(html2, "lxml")
                try:
                    res2 = soup.select('.clearfix')[0].get_text()
                    title = res2.split("\n")[2]
                except IndexError:
                    title = u'好开心么么哒 %s' % random.randint(1, 9999)
                q.put([vurl, title])
        page_q.task_done()
 
def main():
    # 使用帮助
    keyword = input(r"请输入关键字：")
    threads = int(input(u"请输入线程数量(建议1-10): "))
    # 判断目录
    path = '/home/chenwen/mycode/tmp/%s' % keyword
    if os.path.exists(path) == False:
        os.makedirs(path)
    # 解析网页
    logger.info(u"开始解析网页")
    for page in range(1,10):
        page_q.put(page)
    for y in range(threads):
        t = threading.Thread(target=get_page,args=(keyword,page_q))
        t.setDaemon(True)
        t.start()
    page_q.join()
    logger.info(u"共 %s 视频" % q.qsize())
    # 多线程下载
    logger.info(u"开始下载视频")
    for x in range(threads):
        t = threading.Thread(target=downlaod,args=(q,x,path))
        t.setDaemon(True)
        t.start()
    q.join()
    logger.info(u"全部视频下载完成！")
main()


请输入关键字：haha
请输入线程数量(建议1-10): 1
2019-08-29 17:27:02,801 INFO : 开始解析网页


KeyboardInterrupt: 